In [1]:
from __future__ import division
import datetime as dt
from collections import OrderedDict
import sys, os
import dateutil.relativedelta as rd
import json
from pathlib import Path

import utm
import pandas as pd
import numpy as np
import shapely.geometry as sg

DIR = Path('..')
sys.path.append(str(DIR))

import gtfstk as gt

%load_ext autoreload
%autoreload 2

DATA_DIR = DIR/'data'

In [2]:
#path = DATA_DIR/'sample_gtfs.zip'
#path = DATA_DIR/'other_feeds'/'gtfs.zip'
path = DATA_DIR/'cairns_gtfs.zip'

print(gt.list_gtfs(path))

feed = gt.read_gtfs(path, dist_units='km')


            file_name  file_size
0  calendar_dates.txt        387
1          routes.txt       1478
2           trips.txt     143081
3           stops.txt      26183
4          agency.txt        199
5      stop_times.txt    2561019
6          shapes.txt     864694
7        calendar.txt        337


In [9]:
tids = feed.trips.trip_id.iat[0], feed.trips.trip_id.iat[-1]
m = feed.map_trips(tids)
m

In [8]:
m.get_bounds()

[[-16.922937, 145.661993], [-16.741258, 145.780079]]

In [13]:
f = gt.build_null_route_time_series(feed, dates[0], freq='12H', split_directions=True)
f

indicator           num_trip_ends                                          \
route_id                  110-423      110N-423      111-423      112-423   
direction_id                    0    1        0    1       0    1       0   
2014-05-26 00:00:00           NaN  NaN      NaN  NaN     NaN  NaN     NaN   
2014-05-26 12:00:00           NaN  NaN      NaN  NaN     NaN  NaN     NaN   

indicator                             ...  service_duration                    \
route_id                 113-423      ...           142-423      143-423        
direction_id           1       0    1 ...                 0    1       0    1   
2014-05-26 00:00:00  NaN     NaN  NaN ...               NaN  NaN     NaN  NaN   
2014-05-26 12:00:00  NaN     NaN  NaN ...               NaN  NaN     NaN  NaN   

indicator                                                     
route_id            143W-423      150-423      150E-423       
direction_id               0    1       0    1        0    1  
2014-05-26 00:00:00      NaN  NaN     NaN  NaN      NaN  NaN  
2014-05-26 12:00:00      NaN  NaN     NaN  NaN      NaN  NaN  

[2 rows x 220 columns]

In [44]:
#f = feed.compute_stop_time_series(dates[:3], split_directions=True, freq='12H')
#f = feed.compute_route_time_series(trip_stats, dates[:1], freq='12H', split_directions=True)
f = feed.compute_feed_time_series(trip_stats, dates[:1], freq='12H')
#f.columns
pd.isnull(f.values).all()

True

In [ ]:
ts = feed.compute_route_time_series(trip_stats, dates[0], freq='6H')
# inds = ['num_trips', 'num_trip_ends', 'num_trip_starts']
# rids = ['110-423', '111-423', '112-423']

# ts.loc[:, (inds, rids)]

#ts = feed.compute_feed_time_series(trip_stats, [dates[0], dates[2]], freq='6H')
# ts = feed.compute_stop_time_series([dates[0], dates[2]], freq='6H')
# sid = '750015'
# ts.loc[:, ('num_trips', sid)]

ts

In [ ]:
ts.columns

In [ ]:
dts = gt.downsample(ts, freq='D')
#dts.loc[:, (inds, rids)]
#dts.loc[:, ('num_trips', sid)]
dts

In [ ]:
f = feed.compute_feed_stats(trip_stats, '2001')
f


In [ ]:
f.date

In [ ]:
feed.compute_stop_stats(['20140601', '20140603'])

In [ ]:
trip_stats = feed.compute_trip_stats()
feed = feed.append_dist_to_stop_times(trip_stats)

# Load screen line
with (DATA_DIR/'cairns_screen_line.geojson').open() as src:
    line = json.load(src)
    line = sg.shape(line['features'][0]['geometry'])



In [ ]:
f = feed.compute_screen_line_counts(line, dates[:7])
f[f['crossing_time'] < '06:00:00']

In [ ]:
feed.compute_feed_time_series(ts, dates[-2:], freq='12H')

In [ ]:
sd = False
#%time p1 = proto1(feed, ts, dates[:14], split_directions=sd)
%time p2 = proto2(feed, [dates[0], '20010101'], split_directions=sd, freq='12H')
p2

In [ ]:
feed.compute_feed_stats(ts, dates[0])

In [ ]:
feed.describe()

In [ ]:
feed.summarize()